In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os


In [2]:
skin_df = pd.read_csv("/kaggle/input/skindata/skin.csv")

In [3]:
skin_df.head()

,Unnamed: 0,lesion_id,image_id,dx_type,age,sex,localization,image_path,dx,target
0,0,HAM_0000118,ISIC_0027419,histo,80.0,male,scalp,train_all\ISIC_0027419.jpg,bkl,2
1,1,HAM_0000118,ISIC_0025030,histo,80.0,male,scalp,train_all\ISIC_0025030.jpg,bkl,2
2,2,HAM_0002730,ISIC_0026769,histo,80.0,male,scalp,train_all\ISIC_0026769.jpg,bkl,2
3,3,HAM_0002730,ISIC_0025661,histo,80.0,male,scalp,train_all\ISIC_0025661.jpg,bkl,2
4,4,HAM_0001466,ISIC_0031633,histo,75.0,male,ear,train_all\ISIC_0031633.jpg,bkl,2


In [4]:
skin_df['image_id']=skin_df['image_id']+'.jpg'

In [5]:
skin_df.head()

,Unnamed: 0,lesion_id,image_id,dx_type,age,sex,localization,image_path,dx,target
0,0,HAM_0000118,ISIC_0027419.jpg,histo,80.0,male,scalp,train_all\ISIC_0027419.jpg,bkl,2
1,1,HAM_0000118,ISIC_0025030.jpg,histo,80.0,male,scalp,train_all\ISIC_0025030.jpg,bkl,2
2,2,HAM_0002730,ISIC_0026769.jpg,histo,80.0,male,scalp,train_all\ISIC_0026769.jpg,bkl,2
3,3,HAM_0002730,ISIC_0025661.jpg,histo,80.0,male,scalp,train_all\ISIC_0025661.jpg,bkl,2
4,4,HAM_0001466,ISIC_0031633.jpg,histo,75.0,male,ear,train_all\ISIC_0031633.jpg,bkl,2


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(skin_df,test_size=0.20,random_state=1)
train ,val = train_test_split(train,test_size=0.10,random_state=1)

In [7]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    brightness_range=[0.2,1.0],
  # preprocessing_function=myFunc,
    horizontal_flip=True
    )
val_datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
   # preprocessing_function=myFunc,
    brightness_range=[0.2,1.0],
   
    horizontal_flip=True
    )
test_datagen = ImageDataGenerator(
    featurewise_center=False,
    rotation_range=20,
    brightness_range=[0.2,1.0],
  # preprocessing_function=myFunc,
    horizontal_flip=True
    )

Using TensorFlow backend.


In [8]:
train['target']=train['target'].astype('str')
test['target']=test['target'].astype('str')
val['target']=val['target'].astype('str')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
skin_df.head()

,Unnamed: 0,lesion_id,image_id,dx_type,age,sex,localization,image_path,dx,target
0,0,HAM_0000118,ISIC_0027419.jpg,histo,80.0,male,scalp,train_all\ISIC_0027419.jpg,bkl,2
1,1,HAM_0000118,ISIC_0025030.jpg,histo,80.0,male,scalp,train_all\ISIC_0025030.jpg,bkl,2
2,2,HAM_0002730,ISIC_0026769.jpg,histo,80.0,male,scalp,train_all\ISIC_0026769.jpg,bkl,2
3,3,HAM_0002730,ISIC_0025661.jpg,histo,80.0,male,scalp,train_all\ISIC_0025661.jpg,bkl,2
4,4,HAM_0001466,ISIC_0031633.jpg,histo,75.0,male,ear,train_all\ISIC_0031633.jpg,bkl,2


In [10]:
from sklearn.utils import class_weight
 
 
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train['target']),
                                                 train['target'])

In [11]:
class_weights = { i : class_weights[i] for i in range(len(class_weights)) }
class_weights

{0: 4.291666666666667,
 1: 2.7688172043010755,
 2: 1.3121019108280254,
 3: 14.714285714285714,
 4: 1.2638036809815951,
 5: 0.21316225165562913,
 6: 10.729166666666666}

In [12]:
#datagen.flow(train, y_train, batch_size=32)
training_images=train_datagen.flow_from_dataframe(
dataframe=train,
directory="/kaggle/input/images/train_all",
x_col="image_id",
y_col='target',
    color_mode="rgb",
    subset="training",
#     save_to_dir="aug_train",
#     save_format="jpg",
batch_size=32,class_mode="categorical",
target_size=(224,224))#,target_size=(224,224))
#train_datagen.fit(training_images)


val_images=val_datagen.flow_from_dataframe(
dataframe=val,
directory="/kaggle/input/images/train_all",
x_col="image_id",
y_col='target',
    color_mode="rgb",
    subset="training",
#     save_to_dir="aug_val",
#     save_format="jpg",
batch_size=32,
class_mode="categorical",target_size=(224,224))


#val_datagen.fit(val_images)


testing_images=test_datagen.flow_from_dataframe(
dataframe=test,
directory="/kaggle/input/images/train_all",
x_col="image_id",
y_col='target',
    color_mode="rgb",
    subset="training",
#     save_to_dir="aug_test",
#     save_format="jpg",
batch_size=32,
class_mode="categorical",target_size=(224,224))
#test_datagen.fit(testing_images)

Found 7210 validated image filenames belonging to 7 classes.
Found 802 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


In [13]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import ResNet50


In [14]:
base_model= ResNet50(include_top=False, weights="imagenet", input_shape=(224,224,3))

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 2s 0us/step


In [15]:
model= Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Dropout(0.40))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 64)          1179712   
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [17]:
import keras 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['acc'])

In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
filepath="model-{epoch:02d}-{val_acc:.4f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=False, mode='max')
#checkpoint = ModelCheckpoint("vgg16.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [19]:
model.load_weights('/kaggle/input/weights/model-43-0.8779.hdf5')

In [20]:
hist = model.fit_generator(steps_per_epoch=training_images.samples//32,generator=training_images, class_weight=class_weights,validation_data= val_images, validation_steps= val_images.samples//32,epochs=100,callbacks=[early,checkpoint])

Epoch 1/100
225/225 [==============================] - 273s 1s/step - loss: 0.1006 - acc: 0.9352 - val_loss: 0.4611 - val_acc: 0.8525

Epoch 00001: val_acc improved from -inf to 0.85250, saving model to model-01-0.8525.hdf5
Epoch 2/100
225/225 [==============================] - 219s 973ms/step - loss: 0.1063 - acc: 0.9416 - val_loss: 1.0291 - val_acc: 0.8234

Epoch 00002: val_acc did not improve from 0.85250
Epoch 3/100
225/225 [==============================] - 213s 946ms/step - loss: 0.0984 - acc: 0.9464 - val_loss: 0.6027 - val_acc: 0.8286

Epoch 00003: val_acc did not improve from 0.85250
Epoch 4/100
225/225 [==============================] - 211s 937ms/step - loss: 0.1914 - acc: 0.9242 - val_loss: 0.7781 - val_acc: 0.8143

Epoch 00004: val_acc did not improve from 0.85250
Epoch 5/100
225/225 [==============================] - 210s 935ms/step - loss: 0.1295 - acc: 0.9273 - val_loss: 0.2560 - val_acc: 0.7714

Epoch 00005: val_acc did not improve from 0.85250
Epoch 6/100
225/225 [===

In [21]:
model.save_weights('finalweight.h5')

In [22]:
model.save_weights('final.hdf5')

In [24]:
import numpy
test_steps_per_epoch = numpy.math.ceil(testing_images.samples / testing_images.batch_size)

predictions = model.predict_generator(testing_images, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

In [26]:

true_classes = testing_images.classes
class_labels = list(testing_images.class_indices.keys())   

In [27]:
import sklearn.metrics as metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        61
           1       0.05      0.05      0.05        96
           2       0.12      0.07      0.09       228
           3       0.04      0.03      0.03        37
           4       0.13      0.11      0.12       222
           5       0.67      0.76      0.71      1327
           6       0.00      0.00      0.00        32

    accuracy                           0.53      2003
   macro avg       0.15      0.15      0.14      2003
weighted avg       0.48      0.53      0.50      2003



In [31]:
test['target'].value_counts()

5    1327
2     228
4     222
1      96
0      61
3      37
6      32
Name: target, dtype: int64

In [32]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(true_classes, predicted_classes)

ModuleNotFoundError: No module named 'sklearn.metric'